In [6]:
import pandas as pd
from functools import wraps
import random
import urllib
import time
import ssl
import json
import gzip as gz
ssl._create_default_https_context = ssl._create_unverified_context

In [7]:
df = None
with gz.open("data/AllPrintings.json.gz") as f:
    for entry in f:
        data = json.loads(entry)
        df= pd.DataFrame(data)

# print(data.keys())

# df = pd.read_json('./data/cards.json')

In [8]:
df = df.dropna(subset='data')
df.head(10)

,meta,data
10E,NaN,"{'baseSetSize': 383, 'block': 'Core Set', 'boo..."
2ED,NaN,"{'baseSetSize': 302, 'block': 'Core Set', 'boo..."
2X2,NaN,"{'baseSetSize': 577, 'booster': {'default': {'..."
2XM,NaN,"{'baseSetSize': 332, 'booster': {'default': {'..."
30A,NaN,"{'baseSetSize': 594, 'booster': {'default': {'..."
3ED,NaN,"{'baseSetSize': 306, 'block': 'Core Set', 'boo..."
40K,NaN,"{'baseSetSize': 617, 'cards': [{'artist': 'Ant..."
4BB,NaN,"{'baseSetSize': 378, 'cards': [{'artist': 'Har..."
4ED,NaN,"{'baseSetSize': 378, 'block': 'Core Set', 'boo..."
5DN,NaN,"{'baseSetSize': 165, 'block': 'Mirrodin', 'boo..."


In [5]:
df_2 = (
    df
    .assign(**df["data"].apply(pd.Series))
    .explode("cards")
    .loc[:,['cards']]
)

cards = df_2['cards'].to_list()

In [6]:
multiverse_ids = set()
for card in cards:
    if isinstance(card, dict):
        multiverse_ids.add(card.get('identifiers',{}).get('multiverseId'))
    

In [8]:
from concurrent.futures import ThreadPoolExecutor

def threaded(max_workers: int, flatten=False):
    def threaded_func(func):
        @wraps(func)
        def wrapper(partition, *args, **kwargs):
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                futures = {executor.submit(func, row, *args, **kwargs) for row in partition}
            results = [_future.result() for _future in futures]
            if flatten:
                return more_itertools.flatten(results)
            return results

        return wrapper

    return threaded_func

In [9]:

@threaded(12)
def img_down(multiverse_id):
    link = f'https://gatherer.wizards.com/Handlers/Image.ashx?multiverseid={multiverse_id}&type=card'
    name = f'./data/images/{multiverse_id}.jpeg'
    time.sleep(random.random()*10)
    try:
        urllib.request.urlretrieve(link, name, timeout=60)
    except BaseException as err:
        print(err, link)

img_down(multiverse_ids)

<urlopen error [Errno -3] Temporary failure in name resolution> https://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=583818&type=card
<urlopen error [Errno -3] Temporary failure in name resolution> https://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=159751&type=card
<urlopen error [Errno -3] Temporary failure in name resolution> https://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=476301&type=card
<urlopen error [Errno -3] Temporary failure in name resolution> https://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=409758&type=card
<urlopen error [Errno -3] Temporary failure in name resolution> https://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=370620&type=card
<urlopen error [Errno -3] Temporary failure in name resolution> https://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=473221&type=card
<urlopen error [Errno -3] Temporary failure in name resolution> https://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=382908&type=card